In [ ]:
# pip install wandb

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import wandb

from torch.autograd import Variable
from torchvision.models import resnet18, swin_v2_t

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import PIL
import torch
import torchvision

import matplotlib.pyplot as plt
from sklearn import metrics
import tqdm
import json

import os

In [2]:
from PIL import Image
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)
import torchvision.transforms.functional as Function
from IPython.display import display
from time import time
from matplotlib import offsetbox
from sklearn.neighbors import DistanceMetric
%matplotlib inline

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# !rm -rf -r /content/dataset/__MACOSX

In [4]:
def soft_max(array):
    return nn.Softmax(dim=1)(array)

class EmbedNet(nn.Module):
    def __init__(self, base_model, out_dim):
        super(EmbedNet, self).__init__()
        self.base_model = base_model
        self.base_model.fc = torch.nn.Linear(base_model.fc.in_features, 512)
        self.fc1 = torch.nn.Linear(512, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.fc3 = torch.nn.Linear(256, out_dim)

    def forward(self, x):
        x = self.base_model(x)
        x = self.fc1(F.normalize(x))
        x = self.fc2(F.normalize(x))
        x = self.fc3(F.normalize(x))
        return soft_max(F.normalize(x))

In [5]:
data_dir = './data'

# Преобразования для изображений
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Случайное отражение по горизонтали
    transforms.RandomRotation(10),  # Случайное вращение на угол до 10 градусов
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Изменение яркости, контраста, насыщенности и оттенка
    transforms.RandomResizedCrop(224),  # Случайное изменение размера и обрезка изображения до 224x224
    transforms.ToTensor()
])

# Создание датасета из папки с изображениями
dataset = ImageFolder(root=data_dir, transform=transform)

# Разделение на тренировочный и тестовый наборы данных
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

batch_size = 128
# Создание DataLoader для тренировочного и тестового наборов данных
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_to_idx = dataset.class_to_idx
idx_to_class = dict(zip(class_to_idx.values(), class_to_idx.keys()))

In [6]:
# Discard layers at the end of base network
encoder = resnet18(pretrained=True)

out_dim = 7

lr = 0.001

model = EmbedNet(encoder, out_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [9]:
from tqdm import tqdm_notebook

In [10]:
num_epochs = 80

# wandb.init(
#     project="Gagarin-Hack",

#     config={
#     "learning_rate": lr,
#     "batch_size": batch_size,
#     "epochs": num_epochs,
#     }
# )

metrics_dict = {
    'Precision':[],
    'Recall':[],
    'F1-score':[],
    'Accuracy':[],
}

for epoch in range(num_epochs):
    model.train()
    for images, labels in tqdm_notebook(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

    # Оценка модели на тестовом наборе данных
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')
    accuracy = accuracy_score(true_labels, predictions)


    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1-score: {f1:.2f}')
    print(f'Accuracy: {accuracy:.2f}')
    metrics_dict['Precision'].append(precision)
    metrics_dict['Recall'].append(recall)
    metrics_dict['F1-score'].append(f1)
    metrics_dict['Accuracy'].append(accuracy)

    save_model(model, f'./weights/tmp_weights_{epoch}_{precision:.2f}_{recall:.2f}_{f1:.2f}_{accuracy:.2f}.pt')

/var/folders/qn/wj4_7b6s15b8h1mjnbzrfwyryqk4j8/T/ipykernel_78837/3365982749.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for images, labels in tqdm_notebook(train_loader):


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/80, Loss: 1.8630971908569336
Precision: 0.12
Recall: 0.17
F1-score: 0.06
Accuracy: 0.17


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/qn/wj4_7b6s15b8h1mjnbzrfwyryqk4j8/T/ipykernel_78837/3365982749.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for images, labels in tqdm_notebook(train_loader):


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 35/80, Loss: 1.7939090728759766
Precision: 0.67
Recall: 0.52
F1-score: 0.53
Accuracy: 0.52


/var/folders/qn/wj4_7b6s15b8h1mjnbzrfwyryqk4j8/T/ipykernel_78837/3365982749.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for images, labels in tqdm_notebook(train_loader):


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 36/80, Loss: 1.781173825263977


In [ ]:
model.eval()

In [ ]:
img = Image.open('/content/driver_example.jpeg')
transformed_img = transform(img)

In [ ]:
idx_to_class

In [ ]:
model(transformed_img.to(device).unsqueeze(0))

In [ ]:
torch.save(model.state_dict(), 'v1_weights.pt')